### Find annotated calls at a given site

In [ ]:
import os
from marine_acoustics.data_processing import info, read, sample

# Change cwd
os.chdir(r'C:\Users\james\OneDrive - Nexus365\Engineering\Year4\4YP\git\marine_acoustics')


### Choose site and call types

In [ ]:
# ---------------------------
SITE = 9
# ---------------------------
CALLS = [1,2,3]
# ---------------------------

site = info.get_recording_sites()[SITE-1]
call_types = info.get_call_types()
call_types = [call_types[i-1] for i in CALLS]
print(site, '\n', call_types)

In [ ]:
# Folder structure
df_folder_structure = info.get_folder_structure()

# Combine all call-type logs
df_logs = sample.concat_call_logs(site, call_types, df_folder_structure)

# Simplify fields
fields = ['Begin File', 'Beg File Samp (samples)', 'End File Samp (samples)', 'Call Label', 'Begin Date Time', 'Delta Time (s)', 'Low Freq (Hz)',
              'High Freq (Hz)', 'Dur 90% (s)', 'Freq 5% (Hz)', 'Freq 95% (Hz)']
df_logs = df_logs[fields]

# Get default sample rate used at site
_, sr_default = read.read_audio(site, df_logs['Begin File'].iloc[0], df_folder_structure)

# Convert samples to time (s)
df_logs['Beg File Samp (samples)'] = round(df_logs['Beg File Samp (samples)']/sr_default,1)
df_logs['End File Samp (samples)'] = round(df_logs['End File Samp (samples)']/sr_default,1)
df_logs = df_logs.rename(columns={'Beg File Samp (samples)': "Begin (s)", 'End File Samp (samples)': "End (s)"})

# Sort by annotation start time
df_logs = df_logs.sort_values("Begin (s)")

# Groupby .wav filename
gb_wavfile = df_logs.groupby('Begin File')

for wav, logs in gb_wavfile:
    display(logs)